In [ ]:
parameters_path = ""


In [ ]:
from rad import importer

import rad.praise.analysis as pr_analysis



#load the praise objects
(reward_system_objects, distribution_objects) = importer.load_sources_from_json(parameters_path)


# Rewards Analytics and Distribution Dashboard for Quantification Review
This document processes the outputs of the praise reward system and performs an analysis of the resulting token reward distribution. 

## General round summary

In [ ]:
pr_analysis.report_summary.printDescription(distribution_objects['tec_praise_dist'], {})
pr_analysis.report_summary.printGraph(distribution_objects['tec_praise_dist'], {})

# Praise Data Visualization



In [ ]:
pr_analysis.rating_distribution.printDescription(distribution_objects['tec_praise_dist'], {})
pr_analysis.rating_distribution.printGraph(distribution_objects['tec_praise_dist'], {})

### Top 10 highest rated contributions


In [ ]:
#from rad_module.praise.analysis import highest_rated_contributions
#display(Markdown(mod.header))
#display(Markdown(mod.description))
pr_analysis.highest_rated_contributions.printDescription(distribution_objects['tec_praise_dist'], {})
pr_analysis.highest_rated_contributions.printGraph(distribution_objects['tec_praise_dist'], {})

### Praise Reward Distribution

We can now take a look at the distribution of the received praise rewards. You can toggle the inclusion of the different sources by clicking on the legend. 

In [ ]:

'''fig_final_alloc = px.bar(final_token_allocations, x="USER IDENTITY", y = ["QUANT_REWARD", "REWARDBOARD_REWARD", "PRAISE_REWARD"], title="Rewards received by category", color_discrete_map = {'PRAISE_REWARD': 'blue', 'QUANT_REWARD':'green', 'REWARDBOARD_REWARD':'yellow'})
fig_final_alloc.update_xaxes(showticklabels=False)
fig_final_alloc
'''

### Praise Giving Distribution
We can also take a look at the amount of praise different users gave. 


In [ ]:
'''#first we calculate the individual contributions of each praise giver
praise_by_giver = praise_distribution[['FROM USER ACCOUNT', 'AVG SCORE', 'PERCENTAGE', 'TOKEN TO RECEIVE']].copy().groupby(['FROM USER ACCOUNT']).agg('sum').reset_index()
praise_by_giver.rename(columns= {'TOKEN TO RECEIVE': 'TOKENS GAVE'}, inplace = True)
praise_by_giver.sort_values(by='TOKENS GAVE',inplace=True,ascending=False)

fig_praisegiver = px.bar(praise_by_giver, x="FROM USER ACCOUNT",y='TOKENS GAVE',title='Praise Giver Sorted by Total Score')
fig_praisegiver.show()'''

### Praise Flows

Now for something more fun: let's surface the top "praise flows" from the data. Thanks to @inventandchill for this awesome visualization! 
On one side we have the top 15 praise givers separately, on the other the top 25 receivers. The people outside the selection get aggregated into the "REST FROM" and "REST TO" categories.

In [ ]:
# Config specifying how many of the top users to show
#NUMBER_OF_SENDERS: The left side. X largest ones + one bucket for the rest
#NUMBER_OF_RECEIVERS: The right side. Y largest ones + one bucket for the rest 

config = {
    "NUMBER_OF_SENDERS" : 15, 
    "NUMBER_OF_RECEIVERS" : 25 
}

pr_analysis.praise_flow.printDescription(distribution_objects['tec_praise_dist'], {})
pr_analysis.praise_flow.printGraph(distribution_objects['tec_praise_dist'], config)


# Quantifier Data
Now let's take a closer look at the quantification process and the quantifiers:


## Praise Outliers
To aid the revision process, we highlight disagreements between quantifiers.

### Outliers sort by spreads

This graphic visualizes controversial praise ratings by sorting them by the "spread" between the highest and lowest received score. <br>

Please keep in mind that this is a visual aid. If there are several praise instances with similar spread and quant score, all but one end up "hidden" on the chart. For an exhaustive list, take a look at the exported file "praise_outliers.csv" .

In [ ]:
'''col_dismissed = [f'DISMISSED {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
col_dupids = [f'DUPLICATE ID {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
col_scores= [f'SCORE {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]

sort_by_controversial = praise_tools.spread_sort(praise_distribution.copy(), NUMBER_OF_QUANTIFIERS_PER_PRAISE)'''

In [ ]:
'''# Save the outlier table in an external file for easier review
praise_outliers_csv = sort_by_controversial.to_csv(index=False)
with open('praise_outliers.csv', 'w') as f:
    f.write(praise_outliers_csv)'''

In [ ]:
'''# look at what kinda messages get higher spread 
sort_by_controversial['MAX SCORE'] = sort_by_controversial[col_scores].max(axis=1)
spread_df = sort_by_controversial[['REASON','AVG SCORE','MAX SCORE','SPREAD']].sort_values(by='SPREAD')
px.scatter(spread_df,y='MAX SCORE',x='SPREAD',hover_name='REASON')'''

## Praise score by quantifier -- outliers among the quantifiers?

Let's see how different quantifiers behaved by showing the range of praise scores they gave.

To interpret the box plot:

- Bottom horizontal line of box plot is minimum value

- First horizontal line of rectangle shape of box plot is First quartile or 25%

- Second horizontal line of rectangle shape of box plot is Second quartile or 50% or median.

- Third horizontal line of rectangle shape of box plot is third quartile or 75%

- Top horizontal line of rectangle shape of box plot is maximum value.




In [ ]:
'''quant_boxplot = quantifier_rating_table[['QUANT_ID', 'QUANT_VALUE']].copy()
fig_box = px.box(quant_boxplot, x="QUANT_ID", y="QUANT_VALUE", points=False)
fig_box'''

In [ ]:
'''
pr = praise_tools.praise_quantifier(praisedata=praise_distribution,quantifiertable=quantifier_rating_table)
'''

### Score displacement: tendency to under/over-scoring?

In [ ]:
'''fig=pr.plot_mean_displacement()
fig.show()'''

### Scoring correlation: how similiar am I scoring with others?

In [ ]:
'''fig=pr.plot_coefficient()
fig.show()'''

## Agreement on duplication

In [ ]:
'''# pre-process to remove None
dup_agree_check = sort_by_controversial.copy()

for k in range(1,NUMBER_OF_QUANTIFIERS_PER_PRAISE+1):
    col = 'DUPLICATE ID '+str(k)
    dup_agree_check.loc[dup_agree_check[col].isnull(),col]=0

dup_agree_check['DUPLICATION AGREED'] = [len(set(kk))==1 for kk in dup_agree_check[col_dupids].values]'''

In [ ]:
'''# output the duplication disagreement
duplication_disagreement = dup_agree_check.loc[dup_agree_check['DUPLICATION AGREED']!=True,:]

duplication_disagreement = duplication_disagreement.drop(['SPREAD']+col_scores+col_dismissed, axis=1)

md(f'Among **{len(dup_agree_check)}** praises, **{len(duplication_disagreement)} ({len(duplication_disagreement)/len(praise_distribution)*100:.2f}%)** do not agree on duplication')'''

Praise instances with disagreements in duplication are collected in 'results/duplication_examination.csv'. To compare, look at the last 4 columns: 'DUPLICATE MSG 1/2/3' and 'ORIGINAL MSG'.

In [ ]:
'''duplication_disagreement['ORIGINAL MSG']=duplication_disagreement['REASON'] # replicate this column just after the other messages for easy comparison

duplication_disagreement.to_csv('duplication_examination.csv')'''

## Agreement on dismissal

In [ ]:
'''# pre-process to remove None
dism_agree_check = sort_by_controversial.copy()

for k in range(1,NUMBER_OF_QUANTIFIERS_PER_PRAISE+1):
    col = 'DISMISSED '+str(k)
    dism_agree_check.loc[dism_agree_check[col].isnull(),col]=0


dism_agree_check['DISMISSAL AGREED'] = [len(set(kk))==1 for kk in dism_agree_check[col_dismissed].values]

dismiss_disagreement = dism_agree_check.loc[dism_agree_check['DISMISSAL AGREED']==False,:]
dismiss_disagreement= dismiss_disagreement.drop(col_scores+col_dupids+['SPREAD'],axis=1)'''

In [ ]:

'''dismiss_disagreement['ORIGINAL MSG']=dismiss_disagreement['REASON'] # replicate this column just after the other messages for easy comparison

dismiss_disagreement.to_csv('dismissal_disagreed.csv')

md(f'Among **{len(dism_agree_check)}** praises, **{len(dismiss_disagreement)} ({len(dismiss_disagreement)/len(praise_distribution)*100:.2f}%)** do not agree on dismissal')'''



Praise instances with disagreements in dismissal are collected in'results/dismissal_disaggreed.csv'. You can further look into who dismissed and who did not. 

